# LabLivre


## Gather data


In [176]:
# data gathering ran on a separate script

import json
import time

import requests


def get_data(page: int) -> dict:
    url = "https://api.obrasgov.gestao.gov.br/obrasgov/api/projeto-investimento"
    params = {"uf": "DF", "pagina": page, "tamanhoDaPagina": 100}
    headers = {"accept": "*/*"}

    response = requests.get(url, params=params, headers=headers)

    if response.status_code != 200:
        raise Exception(f"Failed to get data: {response.status_code}")

    return response.json()


def main():
    # inelegant but expedient. surely they dont have 10000 items
    for page in range(100):
        response = get_data(page)
        print(f"Page {page} processed")

        # save raw data first so if there are issues, we don't depend on the api
        with open(f"data/data-{page}.json", "w", encoding="utf-8") as f:
            json.dump(response["content"], f, indent=4)
        print(f"Data saved to data/data-{page}.json")

        time.sleep(1)

        page += 1


# if __name__ == "__main__":
#     main()

## Data cleaning


In [177]:
import pandas as pd
import glob


def load_json_files() -> list[dict]:
    """Load and combine all JSON files from data directory into a DataFrame"""
    json_files = glob.glob("data/data-*.json")
    all_records = []

    for file in json_files:
        with open(file, "r") as f:
            records = json.load(f)
            all_records.extend(records)

    return all_records


df = pd.DataFrame(load_json_files())

df.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 834 entries, 0 to 833
Data columns (total 31 columns):
 #   Column                              Non-Null Count  Dtype 
---  ------                              --------------  ----- 
 0   idUnico                             834 non-null    object
 1   nome                                834 non-null    object
 2   cep                                 400 non-null    object
 3   endereco                            430 non-null    object
 4   descricao                           834 non-null    object
 5   funcaoSocial                        834 non-null    object
 6   metaGlobal                          834 non-null    object
 7   dataInicialPrevista                 832 non-null    object
 8   dataFinalPrevista                   832 non-null    object
 9   dataInicialEfetiva                  23 non-null     object
 10  dataFinalEfetiva                    7 non-null      object
 11  dataCadastro                        834 non-null    object

In [178]:
pd.set_option("display.max_columns", None)
df.head()

,idUnico,nome,cep,endereco,descricao,funcaoSocial,metaGlobal,dataInicialPrevista,dataFinalPrevista,dataInicialEfetiva,dataFinalEfetiva,dataCadastro,especie,natureza,naturezaOutras,situacao,descPlanoNacionalPoliticaVinculado,uf,qdtEmpregosGerados,descPopulacaoBeneficiada,populacaoBeneficiada,observacoesPertinentes,isModeladaPorBim,dataSituacao,tomadores,executores,repassadores,eixos,tipos,subTipos,fontesDeRecurso
0,1828.53-16,SISEG - Sistema integrado de segurança,1,,implantação dos sistemas de monitoramento por ...,implantação dos sistemas de monitoramento por ...,Modernização do sistema de segurança do Banco ...,2019-02-04,2024-02-06,None,None,2021-05-12,Recuperação,Projeto,,Cadastrada,None,DF,None,None,None,None,None,2021-05-12,[],"[{'nome': 'BANCO CENTRAL DO BRASIL', 'codigo':...",[],"[{'id': 1, 'descricao': 'Administrativo'}]","[{'id': 5, 'descricao': 'Administrativo', 'idE...","[{'id': 59, 'descricao': 'Obras em Imóveis de ...","[{'origem': 'Federal', 'valorInvestimentoPrevi..."
1,1711.53-18,Reforma do espelho d'água do Edifício-Sede do ...,1,,Reforma do espelho d'água do Edifício-Sede do ...,Reestabelecer a impermeabilização do espelho d...,Recuperação do sistema de impermeabilização e ...,2021-01-04,2021-05-18,None,None,2021-05-04,Reforma,Obra,,Cadastrada,None,DF,None,None,None,None,None,2021-05-04,"[{'nome': 'BANCO CENTRAL DO BRASIL', 'codigo':...","[{'nome': 'BANCO CENTRAL DO BRASIL', 'codigo':...",[],"[{'id': 1, 'descricao': 'Administrativo'}]","[{'id': 5, 'descricao': 'Administrativo', 'idE...","[{'id': 59, 'descricao': 'Obras em Imóveis de ...","[{'origem': 'Federal', 'valorInvestimentoPrevi..."
2,1989.53-01,Elaboração de projetos arquitetônicos e de eng...,1,"Campus Universitário Darcy Ribeiro, Brasília - DF",Contratação de empresa especializada para elab...,Obras para unidades acadêmicas de ensino e pes...,Elaboração de projetos de engenharia e Arquite...,2021-05-18,2021-09-29,None,None,2021-05-17,Construção,Projeto,None,Cadastrada,None,DF,None,None,None,None,None,2021-05-17,[],"[{'nome': 'FUNDACAO UNIVERSIDADE DE BRASILIA',...","[{'nome': 'FUNDACAO UNIVERSIDADE DE BRASILIA',...","[{'id': 1, 'descricao': 'Administrativo'}]","[{'id': 8, 'descricao': 'Educação', 'idEixo': 1}]","[{'id': 46, 'descricao': 'Instituições Federai...","[{'origem': 'Federal', 'valorInvestimentoPrevi..."
3,2004.53-58,SISEG - Sistema integrado de segurança,1,,implantação dos sistemas de monitoramento por ...,implantação dos sistemas de monitoramento por ...,Modernização do sistema de segurança do Banco ...,2019-02-04,2024-02-06,None,None,2021-05-18,Recuperação,Projeto,,Cadastrada,None,DF,None,None,None,None,None,2021-05-18,[{'nome': 'BANCO CENTRAL DO BRASIL-ORC.FISCAL/...,"[{'nome': 'BANCO CENTRAL DO BRASIL', 'codigo':...",[],"[{'id': 1, 'descricao': 'Administrativo'}]","[{'id': 5, 'descricao': 'Administrativo', 'idE...","[{'id': 59, 'descricao': 'Obras em Imóveis de ...","[{'origem': 'Federal', 'valorInvestimentoPrevi..."
4,2617.53-02,CENTRO DE FORMAÇÃO TECNOLOGICA - CFT - CAMPUS ...,1,"Área Especial n° 01, Quadra 16, Cidade do Auto...","construção um galpão com pé direito alto, que ...",atender a comunidade acadêmica do Campus Estru...,aumentar a oferta de cursos técnicos/tecnológi...,2020-12-01,2021-10-27,None,None,2021-06-18,Construção,Obra,,Cadastrada,triplicar as matrículas da educação profission...,DF,None,None,None,None,None,2021-06-18,[{'nome': 'INSTITUTO FED. ED. CIENCIA E TEC. D...,[{'nome': 'INSTITUTO FED. ED. CIENCIA E TEC. D...,"[{'nome': 'MINISTÉRIO DA EDUCAÇÃO', 'codigo': ...","[{'id': 4, 'descricao': 'Social'}]","[{'id': 46, 'descricao': 'Educação', 'idEixo':...","[{'id': 84, 'descricao': 'Educação', 'idTipo':...","[{'origem': 'Federal', 'valorInvestimentoPrevi..."


In [179]:
# use this on each column to check for variable type
df["isModeladaPorBim"].value_counts(dropna=False)

isModeladaPorBim
False    560
None     243
True      31
Name: count, dtype: int64

In [180]:
# categorical variables
df["natureza"] = df["natureza"].astype("category")
df["situacao"] = df["situacao"].astype("category")
df["especie"] = df["especie"].astype("category")
df["uf"] = df["uf"].astype("category")

In [181]:
# date variables
# manually checked fields for formatting errors but found no issues
df["dataInicialPrevista"] = pd.to_datetime(df["dataInicialPrevista"], errors="coerce")
df["dataFinalPrevista"] = pd.to_datetime(df["dataFinalPrevista"], errors="coerce")
df["dataInicialEfetiva"] = pd.to_datetime(df["dataInicialEfetiva"], errors="coerce")
df["dataFinalEfetiva"] = pd.to_datetime(df["dataFinalEfetiva"], errors="coerce")
df["dataCadastro"] = pd.to_datetime(df["dataCadastro"], errors="coerce")
df["dataSituacao"] = pd.to_datetime(df["dataSituacao"], errors="coerce")

In [182]:
# some records have strings instead of ints. these records also have other issues; see below.
# this coerces strings to null
# convert to float instead of int to allow for null values
df["qdtEmpregosGerados"] = pd.to_numeric(df["qdtEmpregosGerados"], errors="coerce")
df["populacaoBeneficiada"] = pd.to_numeric(df["populacaoBeneficiada"], errors="coerce")

In [183]:
df["idLabLivre"] = df.index

### Dealing with nested cols


In [184]:
nested_cols = [
    "tomadores",
    "executores",
    "repassadores",
    "eixos",
    "tipos",
    "subTipos",
    "fontesDeRecurso",
]

In [185]:
# Convert empty lists to None
# for col in nested_cols:
#     df[col] = df[col].apply(lambda x: None if x == [] else x)

In [ ]:
# pd.json_normalize(df['tomadores'].explode()) is the easy way to do this but it converts the id to float, which might cause issues due to floating point imprecision even if converted back to int. because of this, i tried this convoluted solution:

# tomadores_df = df[["idLabLivre", "tomadores"]].explode("tomadores")
# tomadores_df = tomadores_df.dropna(subset=["tomadores"])
# tomadores_df["nome"] = tomadores_df["tomadores"].apply(lambda x: x["nome"])
# tomadores_df["codigo"] = tomadores_df["tomadores"].apply(lambda x: str(x["codigo"]))
# tomadores_df = tomadores_df.drop(columns=["tomadores"])
# tomadores_df

# this does not handle multiple entries in the json object properly. i am tired so i am just going to hope the float conversion doesnt mess up the ids. the names should still work properly.

,idLabLivre,nome,codigo
1,1,BANCO CENTRAL DO BRASIL,25280
3,3,BANCO CENTRAL DO BRASIL-ORC.FISCAL/SEG.SOCIAL,25201
4,4,INSTITUTO FED. ED. CIENCIA E TEC. DE BRASILIA,26356
6,6,MINISTÉRIO DE MINAS E ENERGIA,2852
8,8,DEPARTAMENTO DE POLICIA FEDERAL,30108
...,...,...,...
823,823,FUNDACAO UNIVERSIDADE DE BRASILIA,26271
827,827,SUPERINTENDENCIA DA POLICIA RODOVIÁRIA FEDERAL...,394494013629
828,828,FUNDACAO UNIVERSIDADE DE BRASILIA,26271
829,829,Polícia Militar do Distrito Federal,8942610000116


In [ ]:
# Check number of unique values for object columns
for col in df.select_dtypes(include=["object"]).columns:
    n_unique = df[col].nunique()
    print(f"{col}: {n_unique} unique values")

In [196]:
# Check for duplicates based on 'idUnico'
n_duplicates = df["idUnico"].duplicated().sum()
print(f"Number of duplicates by idUnico: {n_duplicates}")

if n_duplicates > 0:
    # Show examples of duplicate records
    duplicate_ids = df[df["idUnico"].duplicated(keep=False)]["idUnico"].unique()
    print("\nExample duplicate records:")
    for id_unico in duplicate_ids[:3]:  # Show up to 3 examples
        print(f"\nRecords with idUnico '{id_unico}':")
        print(
            df[df["idUnico"] == id_unico][
                ["idUnico", "nome", "dataCadastro", "situacao"]
            ].to_string()
        )

Number of duplicates by idUnico: 122

Example duplicate records:

Records with idUnico '3938.53-53':
        idUnico                                                                         nome dataCadastro    situacao
95   3938.53-53  Remanescentes da obra de acessibilidade de demais adequações do Campus Gama   2021-09-23  Cadastrada
306  3938.53-53  Remanescentes da obra de acessibilidade de demais adequações do Campus Gama   2021-09-23  Cadastrada

Records with idUnico '44063.53-80':
         idUnico                                                         nome dataCadastro    situacao
100  44063.53-80  Regularização Fundiária do Sol Nascente, Trecho II, Etapa 1   2024-10-21  Cadastrada
535  44063.53-80  Regularização Fundiária do Sol Nascente, Trecho II, Etapa 1   2024-10-21  Cadastrada

Records with idUnico '58004.53-70':
         idUnico                                                                                             nome dataCadastro    situacao
101  58004.53-70  Const

In [ ]:
# todo manually check if these duplicates are exactly the same and if so drop.

## Load DB


In [ ]:
import sqlite3

con = sqlite3.connect("projeto_investimento.db")
df.to_sql("projeto_investimento", con, if_exists="replace")

## Analysis


In [ ]:
# todo
# check map
# check dates compared to today

I dont know what these data mean so I chose not to drop any null values.


## Insights


-   not sure how this is collected but `descPlanoNacionalPoliticaVinculado` is clearly confusing for whoever is filling this form. some treat as categorical and some as a text field.
-   some entries have systematic issues, with `descPopulacaoBeneficiada` being template text and `qdtEmpregosGerados` as empty string: `df[(df["qdtEmpregosGerados"] == ' ') & (df['descPopulacaoBeneficiada'] == 'Descrição população beneficiada')]`
    -   warrants further investigation; ask a domain expert.
